In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
# pip install selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [79]:
# initialize our browser
# need to change to your own path
path = "/Users/siang-linghsu/opt/anaconda3/share/jupyter/lab/chromedriver"


browser = webdriver.Chrome(path)

# go the specific hotel page

link = "https://www.tripadvisor.in/Hotel_Review-g60763-d93486-Reviews-Millennium_Hotel_Broadway_Times_Square-New_York_City_New_York.html"
browser.get(link)

time.sleep(2)


In [3]:
# a helper function to check if the button exists on the page
def check_exists_by_xpath(xpath):
    try:
        browser.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [4]:
# let's make it a helper function too
def click_read_more():
    read_more_exists = check_exists_by_xpath('//span[@class="ui_icon caret-down zedZf"]')
   
    if read_more_exists:
        element = browser.find_element_by_xpath('//span[@class="ui_icon caret-down zedZf"]')
        browser.execute_script("arguments[0].click();", element)
        time.sleep(5)

In [85]:
# back to the first page:
browser.find_element_by_xpath('//*[@id="component_15"]/div/div[3]/div/div[8]/div/div/a[1]').click()


In [97]:
#change page number
page_num = 10

#reviews_title = []
#reviews_text = []
#ratings = []
#authors= []
#stay_dates= []

for i in range(0, page_num):
    
    # expand the reviews
    click_read_more() 
    
    
    # parse to a soup
    page_source = browser.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    reviews_content = soup.find_all('div', class_='cWwQK MC R2 Gi z Z BB dXjiy')
    time.sleep(3)
    # find all find every reviews
    
    # extract the author, review_text and rating 
    for review in reviews_content:
        
        author= review.find('div',class_ = 'bcaHz').a.text
        review_title = review.find('div',class_ = 'fpMxB MC _S b S6 H5 _a').text
        review_text = review.find('div', class_='pIRBV _T').text
        rating = review.find('div', class_='emWez F1').span['class'][1][-2]
        stay_date = review.find('span',class_='euPKI _R Me S4 H3')
        # append to our accumulative lists
        reviews_title.append(review_title)
        reviews_text.append(review_text)
        ratings.append(rating)
        authors.append(author)
        try:
            stay_dates.append(stay_date.text[14:])
        except Exception:
            stay_dates.append('N/A')
        
    # use selenium to go to the next page
    if (check_exists_by_xpath('//a[@class="ui_button nav next primary "]')):
        browser.find_element_by_xpath('//a[@class="ui_button nav next primary "]').click()
        time.sleep(2)


In [98]:
# to check the length of each attritube

print(len(reviews_title))
print(len(reviews_text))
print(len(ratings))
print(len(authors))
print(len(stay_dates))




2020
2020
2020
2020
2020


In [99]:
# store the data into a dataset
data = {
    'reviews_title':reviews_title,
    'reviews_text' : reviews_text,
    'ratings' :ratings,
    'author':authors,
    'stay_date':stay_dates
}

df = pd.DataFrame(data)

# check the shape of our dataframe
df.shape


(2020, 5)

In [88]:
df.head()

,reviews_title,reviews_text,ratings,author,stay_date
0,DON'T FALL FOR THE UPGRADE SCAM,1st off hotel is in a brilliant location on ti...,1,Junior A,December 2021
1,Eva's 9th bday trip,"Daniel was awesome at the front desk, the hote...",5,Paul N,June 2021
2,Not good at all…,"Oh no, this place is really not great. Locatio...",1,Jack,November 2021
3,BEST Hotel in Broadway Times Square!!!,I cannot say enough great things about the Mil...,5,Stephanie C,November 2021
4,We arrived at the hotel from the airport WITH ...,When we arrived at the hotel from the airport ...,1,Lauren W,December 2021


In [100]:
# write to a csv file
# need to change file name
df.to_csv('hotel_Millennium.csv')

In [46]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [102]:

#reviews_title
#reviews_text
#ratings
#authors
#stay_dates
# remove number and punctuation
import re
import nltk
review_text_without_punctuations = []


for review in reviews_text:
    review_text_without_punctuations.append( re.sub('[^a-zA-Z ]+', '', str(review)))
    
    
# tokenize
nltk.download('punkt')
from nltk.tokenize import word_tokenize
review_text_tokenizing = []
for review in review_text_without_punctuations:
    review_text_tokenizing.append(word_tokenize(review))

# remove stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk_stop_words = stopwords.words('english')

review_text_without_stopword = []

for review in review_text_tokenizing :
    for text in review:
        if text.lower() not in nltk_stop_words :
            review_text_without_stopword.append(text)
        
# stemming
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
reviews_text_after_stemming = []

for review in review_text_without_stopword:
    reviews_text_after_stemming.append(stemmer.stem(review))
    
    


[nltk_data] Downloading package punkt to /Users/siang-
[nltk_data]     linghsu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/siang-
[nltk_data]     linghsu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
# POS TAGGING
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
review_pos_tagging = pos_tag(reviews_text_after_stemming)
#review_pos_tagging


# Extracting all Nouns from a text file using nltk

#for i in range(0,1):
#    print( [(word, tag) for word, tag in review_pos_tagging if (tag=='NN')])



nouns = [word for word,pos in review_pos_tagging \
            if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/siang-linghsu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# TF-IDF MATRIX
from sklearn.feature_extraction.text import TfidfVectorizer
reviews_text_after_stemming

In [93]:
nouns

['mention',
 'rommel',
 'gopez',
 'hotel',
 'edison',
 'year',
 'help',
 'advis',
 'group',
 'visit',
 'age',
 'covid',
 'relief',
 'feel',
 'welcomei',
 'hotel',
 'edison',
 'friend',
 'nd',
 'rd',
 'signatur',
 'room',
 'stay',
 'comfort',
 'staff',
 'help',
 'hotel',
 'liter',
 'minut',
 'walk',
 'time',
 'squar',
 'bar',
 'york',
 'rum',
 'situat',
 'lobbi',
 'mai',
 'tai',
 'fantast',
 'cant',
 'wait',
 'group',
 'friend',
 'visit',
 'hotel',
 'edison',
 'easi',
 'staff',
 'help',
 'concierg',
 'room',
 'personnel',
 'cours',
 'staff',
 'edison',
 'quit',
 'simpli',
 'look',
 'staff',
 'hotel',
 'edison',
 'front',
 'desk',
 'housekeep',
 'shout',
 'justina',
 'tex',
 'jennif',
 'marco',
 'kind',
 'hospit',
 'luidin',
 'manag',
 'staff',
 'hotel',
 'room',
 'find',
 'hotel',
 'stay',
 'time',
 'squar',
 'shout',
 'ladi',
 'virginia',
 'grissel',
 'erick',
 'danni',
 'job',
 'pleasur',
 'day',
 'custom',
 'person',
 'trip',
 'enjoy',
 'place',
 'environ',
 'stay',
 'day',
 'room',


In [44]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk_stop_words = stopwords.words('english')

review_text_wthout_stopword = [i for i in review_text_without_punctuations if i not in stopwords]
print(review_text_wthout_stopword)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/siang-linghsu/nltk_data'
    - '/Users/siang-linghsu/opt/anaconda3/nltk_data'
    - '/Users/siang-linghsu/opt/anaconda3/share/nltk_data'
    - '/Users/siang-linghsu/opt/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [37]:
print(reviews_text)

['I would like to mention Rommel Gopez. I have stayed at Hotel Edison before, but it was quite a few years ago. Rommel was so helpful in advising our group prior to and during our visit. In this age of Covid, it was such a relief to feel both safe and welcome.I will be sure to recommend Hotel Edison to all my friends.', 'This was our 2nd stay at The Edison and we will be sure to have a 3rd. The signature room we stayed in was clean and comfortable and the staff are always polite and helpful. The hotel is literally just one minute walk to Times Square and the best bar in New York, The Rum House, is situated in the lobby. The Mai Tai’s there are fantastic! Can’t wait to go again.', 'Our group of friends enjoyed our recent visit to NYC.  Staying at the Hotel Edison makes it so easy to go to the Broadway shows we were looking forward to.  You cannot beat the location!  The staff is incredible.  From the very helpful concierge to the room service personnel, and, of course, to our friend Rom